<a href="https://colab.research.google.com/github/hoseinm444/FanapExercises/blob/main/tmdb_telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#from contextlib import nullcontext
#!pip install themoviedb
#!pip install python-telegram-bot

from themoviedb import TMDb
from typing import Final
from telegram import Update ,KeyboardButton,InlineKeyboardButton,InlineKeyboardMarkup ,ReplyKeyboardMarkup,Bot
from telegram.ext import Application , CommandHandler,CallbackQueryHandler , MessageHandler, ContextTypes , filters
import nest_asyncio
nest_asyncio.apply()

token:Final = ''
Bot_username:Final = ''
tmdb = TMDb(key="", language="En", region="En")

#command
#start command
async def start_command(update: Update,context:ContextTypes.DEFAULT_TYPE):
  #when user hit start our response is so:
  print("start button..")#for debugging
  await update.message.reply_text(text="Welcome to YourBot! Please Enter your desire movie name...")

#help command
async def help_command(update: Update,context:ContextTypes.DEFAULT_TYPE):
  #tell user how the bot works :
  print("help button..") #for debugging
  await update.message.reply_text("Enter your desire movie name and we will show you a list of movies according to it.")

#search command
async def search_command(update: Update,context:ContextTypes.DEFAULT_TYPE) :
    print("search button..\n") #for debugging
    await update.message.reply_text("Please Enter your movie name :")

def search_for_movie(text:str):
    processed : str =text.lower()
    print(f'search for : {processed}')#for debugging
    items = tmdb.search( ).movies(processed)
    dictionary = {}
    print(f'Movie Itmes : {items}\n')
    print(f'Movie Itmes : {items.total_results} , total_Pages:{items.total_pages},current_page:{items.page}\n')
    for i in items:
        movie_id = i.id
        movie = tmdb.movie(movie_id).details(append_to_response="credits,external_ids,images,videos")
        dictionary.__setitem__(movie.id, movie.title)
    return dictionary
    if(items.total_results == 0):
      return 'we can not find such movie or your movie name is incorrect..'

async def handle_message(update: Update,context:ContextTypes.DEFAULT_TYPE):
    print('resopnse  is :')
    processed : str = update.message.text
    dictionary = search_for_movie(processed)
    if len(dictionary)>0:
      for key , value in dictionary.items():
           keyboard = [
                    [
                         InlineKeyboardButton("detail", callback_data=f'{key}'),
                         #InlineKeyboardButton("imdb url", callback_data=f'{value}'),
                    ],
                 ]

           reply_markup = InlineKeyboardMarkup(keyboard)
           await update.message.reply_text(f"{value}:", reply_markup=reply_markup)
    else :
            await update.message.reply_text('your movie name is not found ')

async def button(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Parses the CallbackQuery and updates the message text."""
    query = update.callback_query
    print(f'query :{query}')
    print(f'query :{type(query.data)}')
    try:
      id_query_int = int(query.data)
    except error as e:
      print(e)
    movie = tmdb.movie(id_query_int).details(append_to_response="people,credits,external_ids,images,videos,person")
    gener_list =[]
    companies_list= []
    main_character=[]
    countries_list =[]
    for i in movie.credits.cast:
      main_character.append(i.name)
    print(main_character)
    for i in movie.production_companies:
       companies_list.append(i.name)
    for i in movie.genres:
      gener_list.append(i.name)
    for i in movie.production_countries:
      countries_list.append(i.name)
    movie_dict =[
                  f'title : {movie.title}',
                  f'year : {movie.year}' ,
                  f'popularity : {movie.popularity}' ,
                  f'imdb url : {movie.imdb_url}' ,
                  f'vote : {movie.vote_average}',
                  f'overview : {movie.overview}' ,
                  f'genres :{gener_list}',
                  f'companies : {companies_list}',
                  f'countries : {countries_list}',
                  f'characters :{main_character}' #make the text too long
               ]
    nl = '\n'
    print(movie_dict)
    # CallbackQueries need to be answered, even if no notification to the user is needed
    # Some clients may have trouble otherwise. See https://core.telegram.org/bots/api#callbackquery
    await query.answer()

    await query.edit_message_text(text=f"movie is : {nl}{nl.join(movie_dict)}")

#logging errors
async def error(update: Update,context:ContextTypes.DEFAULT_TYPE):
  print('error....')
  print(f'Update {update} caused error {context.error}')


if __name__ == '__main__' :
   # this for debugging
   print('starting bot...')
   app = Application.builder().token(token).build()

   # Buttoms


   # Commands
   app.add_handler(CommandHandler('start',start_command))
   app.add_handler(CommandHandler('help',help_command))
   app.add_handler(CommandHandler('search',search_command))
   app.add_handler(CallbackQueryHandler(button))

   # Messages
   app.add_handler(MessageHandler(filters.TEXT,handle_message))

   # Errors
   app.add_error_handler(error)
     # check for new message constantly  - polls the bot
   print('Polling....')
   app.run_polling(poll_interval=3)


starting bot...
Polling....
resopnse  is :
search for : owl
Movie Itmes : Movies(results=[PartialMovie(id=660624, poster_path='/qRQ5YlRh9PnKxc9qRTebPA9q5kh.jpg', adult=False, overview='His trainer stands below the tree holding a dead mouse by its tail and a clicker. Dimitri is meant to fly to his trainer. He has missed his cue. A perfectly still film crew is waiting for the owl to make his move. The crew has been shooting in these woods all week and has spotted a baby eagle and it’s mother. Dimitri seems to sense the eagle’s presence. The trainer calls the owl with a whistle and clicks his clicker. Dimitri is not listening.', release_date=datetime.date(2019, 12, 1), genre_ids=[], original_title='Owl', original_language='en', title='Owl', backdrop_path=None, popularity=0.639, vote_count=0, video=False, vote_average=0.0, media_type=<MediaType.movie: 'movie'>), PartialMovie(id=40975, poster_path='/lg7veUKM3VqSnu0bl8otEnDs7M4.jpg', adult=False, overview="Poor Bibi - due to her bad marks in

RuntimeError: Cannot close a running event loop